# Relabel a CSV file

This notebook is used to relabel a CSV file of gestures. When gesture data gets recorded, labels are automatically added at regular intervals, but the actual timing of when the user's hands move might not align perfectly with those automatically added labels. This notebook can be run and will visualise the data in the CSV file along with the labels. The CSV file can then be edited externally and reloaded in this notebook. In this way, the labels can (slowly) be moved around until they align with the gestures.

In [ ]:
# Change directory to keep paths consistent
%cd /Users/brk/projects/masters/SU/ergo/src

In [ ]:
# Imports and setup
%load_ext autoreload
%autoreload 2
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
import pandas as pd
import numpy as np
import models
import vis
import common
import read
import tensorflow as tf
from tensorflow import keras
from keras import layers
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tqdm
import yaml
import glob
import re

# Visualise a CSV file of data

In [ ]:
filename = "../gesture_data/new_data/2023-08-20T13:35:37.csv"
const: common.ConstantsDict = common.read_constants('../src/constants.yaml')
names = ['datetime', 'gesture'] + list(const["sensors"].values())

In [ ]:

@interact(start=(0, 14185, 20))
def fn(start=0):
    duration = 120
    t0 = start
    t1 = start + duration
    df = pd.read_csv(filename, header=None, names=names)
    values = df.loc[t0:t1, names[2:]].values
    labels = df.loc[t0:t1, 'gesture'].values
    datetimes = df.loc[t0:t1, 'datetime'].values
    ylim = values.min()*0.9, values.max()*1.1
    mid = (ylim[1] - ylim[0]) * 0.5
    # NOTE: this only visualises the right hand
    fig, axs = plt.subplots(5, 1, figsize=(10, 10))
    for i, ax in enumerate(axs, 5):
        ax.plot(values[:, i*3+0], c='tab:red')
        ax.plot(values[:, i*3+1], c='tab:green')
        ax.plot(values[:, i*3+2], c='tab:blue')

        ax.scatter(range(values.shape[0]), values[:, i*3+0], c='tab:red', s=3)
        ax.scatter(range(values.shape[0]), values[:, i*3+1], c='tab:green', s=3)
        ax.scatter(range(values.shape[0]), values[:, i*3+2], c='tab:blue', s=3)
        for x, label in filter(lambda l: l[1] != 'gesture0255', enumerate(labels)):
            ax.text(x, ylim[1], label.replace('gesture00', 'g'), rotation=90, va='top')
        ax.set(
            yticks=[],
            xticks=[],
            ylim=ylim,
            ylabel=f'{i}: ' + ",".join(names[2:][i*3:i*3+3]),
        )
        if i in (4, 9):
            xticklabels = [(None if labels[i] == 'gesture0255' else dt[11:-4]) for i, dt in enumerate(datetimes)]
            ax.set_xticks(range(len(xticklabels)))
            ax.set_xticklabels(xticklabels, rotation=90)
    plt.tight_layout()